In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# hyperparameters
block_size = 64
batch_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100

In [9]:
text = open('../data/shakespeare.txt', 'r', encoding='utf-8').read()

In [17]:
chars = sorted(list(set(text)))

# create character to token mapping
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }
encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [67]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [68]:
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

(tensor([[60, 43, 42,  ..., 47, 58,  1],
         [ 1, 58, 46,  ...,  0, 16, 56],
         [44,  1, 52,  ..., 58, 46, 53],
         ...,
         [13, 10,  0,  ..., 53,  1, 61],
         [ 0,  0, 18,  ..., 46, 63, 57],
         [50,  1, 50,  ...,  1, 51, 39]]),
 tensor([[43, 42,  1,  ..., 58,  1, 58],
         [58, 46, 53,  ..., 16, 56, 47],
         [ 1, 52, 53,  ..., 46, 53, 59],
         ...,
         [10,  0, 13,  ...,  1, 61, 43],
         [ 0, 18, 47,  ..., 63, 57, 43],
         [ 1, 50, 47,  ..., 51, 39, 63]]))

In [70]:
@torch.no_grad()
def estimate_loss(model):
    model.eval()
    out = {}
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        # compute attention scores (affinities)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # values
        v = self.value(x)
        out = wei @ v
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size, n_head):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in n_head])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd)
            nn.ReLU()
            nn.Linear(4*n_embd, n_embd)
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(head_size, n_head)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return x